# Regressão

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

No capítulo anterior vimos vários exemplos de regressão logística, que se baseia no pressuposto de que a probabilidade de um resultado, expressa sob a forma de probabilidades logísticas, é uma função linear de alguma quantidade (contínua ou discreta).

Neste capítulo vamos trabalhar em exemplos de regressão linear simples, que modelam a relação entre duas quantidades.  Especificamente, vamos analisar as mudanças ao longo do tempo na queda de neve e o recorde mundial da maratona.

Os modelos que utilizaremos têm três parâmetros, pelo que poderá querer rever as ferramentas que utilizámos para o modelo de três parâmetros em <<_MarkandRecapture>>>>.

## Mais neve?

Tenho a impressão de que não temos tanta neve por aqui como costumávamos ter.  Por "por aqui" quero dizer o condado de Norfolk, Massachusetts, onde nasci, cresci, e actualmente vivo.  E por "costumava" quero dizer comparado com quando era jovem, como em 1978, quando recebemos [27 inches of snow](https://en.wikipedia.org/wiki/Northeastern_United_States_blizzard_of_1978) e não tive de ir à escola durante algumas semanas.

Felizmente, podemos testar a minha conjectura com dados.  Acontece que o Condado de Norfolk é a localização do [Blue Hill Meteorological Observatory](https://en.wikipedia.org/wiki/Blue_Hill_Meteorological_Observatory), que mantém o registo meteorológico contínuo mais antigo da América do Norte.

Os dados desta e de muitas outras estações meteorológicas estão disponíveis no [National Oceanic and Atmospheric Administration](https://www.ncdc.noaa.gov/cdo-web/search) (NOAA).  Recolhi dados do Observatório de Blue Hill de 11 de Maio de 1967 a 11 de Maio de 2020.  

A célula seguinte descarrega os dados como um ficheiro CSV.

In [4]:
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/data/2239075.csv')

Podemos utilizar Pandas para ler os dados em `DataFrame':

In [5]:
import pandas as pd

df = pd.read_csv('2239075.csv', parse_dates=[2])

Eis como se parecem as últimas filas.

In [6]:
df.tail(3)

As colunas que vamos utilizar são:

*DATE', que é a data de cada observação,

*`SNOW`, que é a queda total de neve em centímetros.

Vou acrescentar uma coluna que contém apenas a parte do ano das datas.

In [7]:
df['YEAR'] = df['DATE'].dt.year

E utilizar `agrupar por` para somar a queda de neve total em cada ano.

In [8]:
snow = df.groupby('YEAR')['SNOW'].sum()

O primeiro e último anos não estão completos, por isso vou deixá-los cair.

In [9]:
snow = snow.iloc[1:-1]
len(snow)

A figura seguinte mostra a queda total de neve durante cada um dos anos completos da minha vida.

In [10]:
from utils import decorate

snow.plot(style='o', alpha=0.5)

decorate(xlabel='Year',
         ylabel='Total annual snowfall (inches)',
         title='Total annual snowfall in Norfolk County, MA')

Olhando para esta trama, é difícil dizer se a queda de neve está a aumentar, a diminuir, ou a permanecer inalterada.  Na última década, tivemos vários anos com mais neve que 1978, incluindo 2015, que foi o inverno mais nevado da área de Boston na história moderna, com um total de 141 polegadas.

Este tipo de pergunta - olhando para dados ruidosos e perguntando-se se está a subir ou a descer - é precisamente a pergunta a que podemos responder com a regressão Bayesiana.

In [11]:
snow.loc[[1978, 1996, 2015]]

## Modelo de regressão

A base da regressão (Bayesiana ou não) é a suposição de que uma série temporal como esta é a soma de duas partes:

1. Uma função linear do tempo, e

2. Uma série de valores aleatórios retirados de uma distribuição que não está a mudar com o tempo.

Matematicamente, o modelo de regressão é

$$y = a x + b + \epsilon$$

onde $y$ é a série de medições (queda de neve neste exemplo), $x$ é a série de vezes (anos) e $\silon$ é a série de valores aleatórios.

$a$ e $b$ são o declive e a intercepção da linha através dos dados.  São parâmetros desconhecidos, pelo que utilizaremos os dados para os estimar.

Não sabemos a distribuição de $\silon$, por isso vamos fazer a suposição adicional de que é uma distribuição normal com média 0 e desvio padrão desconhecido, $\sigma$.  

Para ver se esta hipótese é razoável, vou traçar a distribuição da queda total de neve e um modelo normal com a mesma média e desvio padrão.

Aqui está um objecto `Pmf` que representa a distribuição da queda de neve.

In [12]:
from empiricaldist import Pmf

pmf_snowfall = Pmf.from_seq(snow)

E aqui estão a média e o desvio padrão dos dados.

In [13]:
mean, std = pmf_snowfall.mean(), pmf_snowfall.std()
mean, std

Vou utilizar o objecto `norm` da SciPy para calcular o CDF de uma distribuição normal com a mesma média e desvio padrão.

In [14]:
from scipy.stats import norm

dist = norm(mean, std)
qs = pmf_snowfall.qs
ps = dist.cdf(qs)

Eis como se parece a distribuição dos dados em comparação com o modelo normal.

In [15]:
import matplotlib.pyplot as plt

plt.plot(qs, ps, color='C5', label='model')
pmf_snowfall.make_cdf().plot(label='data')

decorate(xlabel='Total snowfall (inches)',
         ylabel='CDF',
         title='Normal model of variation in snowfall')

Tivemos mais Invernos abaixo da média do que o esperado, mas no geral isto parece ser um modelo razoável.

## Regressão de mínimos quadrados

O nosso modelo de regressão tem três parâmetros: inclinação, intercepção, e desvio padrão de $\epsilon$.

Antes de os podermos estimar, temos de escolher os seus antecedentes.

Para ajudar nisso, vou utilizar o StatsModel para ajustar uma linha aos dados por [least squares regression](https://en.wikipedia.org/wiki/Least_squares).

Primeiro, vou utilizar o `reset_index' para converter o `snow', que é uma `Série', para um `DataFrame'.

In [16]:
data = snow.reset_index()
data.head(3)

O resultado é um `DataFrame` com duas colunas, `YEAR` e `SNOW`, num formato que podemos utilizar com StatsModels.

Como fizemos no capítulo anterior, centrarei os dados, subtraindo a média.

In [17]:
offset = data['YEAR'].mean().round()
data['x'] = data['YEAR'] - offset
offset

E vou acrescentar uma coluna aos "dados" para que a variável dependente tenha um nome padrão.

In [18]:
data['y'] = data['SNOW']

Agora, podemos utilizar StatsModels para calcular os quadrados mínimos adequados aos dados e estimar "inclinação" e "intercepção".

In [19]:
import statsmodels.formula.api as smf

formula = 'y ~ x'
results = smf.ols(formula, data=data).fit()
results.params

A intercepção, cerca de 64 polegadas, é a queda de neve esperada quando `x=0`, que é o início de 1994.

A inclinação estimada indica que a queda total de neve está a aumentar a uma taxa de cerca de 0,5 centímetros por ano.  

Os "resultados" também fornecem "resid", que é um conjunto de resíduos, ou seja, as diferenças entre os dados e a linha instalada.

O desvio padrão dos resíduos é uma estimativa do `sigma`.

In [20]:
results.resid.std()

Utilizaremos estas estimativas para escolher distribuições prévias para os parâmetros.

## Priores

Vou utilizar distribuições uniformes para os três parâmetros.

In [21]:
import numpy as np
from utils import make_uniform

qs = np.linspace(-0.5, 1.5, 51)
prior_slope = make_uniform(qs, 'Slope')

In [22]:
qs = np.linspace(54, 75, 41)
prior_inter = make_uniform(qs, 'Intercept')

In [23]:
qs = np.linspace(20, 35, 31)
prior_sigma = make_uniform(qs, 'Sigma')

Fiz as distribuições anteriores em comprimentos diferentes por duas razões.  Primeiro, se cometermos um erro e usarmos a distribuição errada, será mais fácil apanhar o erro se todos os comprimentos forem diferentes.

Em segundo lugar, fornece mais precisão para o parâmetro mais importante, "inclinação", e gasta menos esforço computacional no parâmetro menos importante, "igma".

Em <<_ThreeParameterModel>> fizemos uma distribuição conjunta com três parâmetros.  Envolverei esse processo numa função:

In [24]:
from utils import make_joint

def make_joint3(pmf1, pmf2, pmf3):
    """Make a joint distribution with three parameters."""
    joint2 = make_joint(pmf2, pmf1).stack()
    joint3 = make_joint(pmf3, joint2).stack()
    return Pmf(joint3)

E utilizá-lo para fazer um `Pmf` que representa a distribuição conjunta dos três parâmetros.

In [25]:
prior = make_joint3(prior_slope, prior_inter, prior_sigma)
prior.head(3)

O índice de `Pmf` tem três colunas, contendo valores de `slope`, `inter`, e `sigma`, nessa ordem.

Com três parâmetros, o tamanho da distribuição conjunta começa a ficar grande.  Especificamente, é o produto dos comprimentos das distribuições anteriores.  Neste exemplo, as distribuições anteriores têm 51, 41, e 31 valores, pelo que o comprimento da distribuição anterior da junta é de 64.821.

In [26]:
len(prior_slope), len(prior_inter), len(prior_sigma)

In [27]:
len(prior_slope) * len(prior_inter) * len(prior_sigma)

In [28]:
len(prior)

## Probabilidade

Agora vamos calcular a probabilidade dos dados.

Para demonstrar o processo, vamos assumir temporariamente que os parâmetros são conhecidos.

In [29]:
inter = 64
slope = 0.51
sigma = 25

Vou extrair os `xs` e `ys` dos `dados` como objectos `Série`:

In [30]:
xs = data['x']
ys = data['y']

E calcular os "resíduos", que são as diferenças entre os valores reais, "ys", e os valores que esperamos baseados em "declive" e "inter".

In [31]:
expected = slope * xs + inter
resid = ys - expected

De acordo com o modelo, os resíduos devem seguir uma distribuição normal com média 0 e desvio padrão `sigma`.  Assim, podemos calcular a probabilidade de cada valor residual utilizando o `norm` da SciPy.

In [32]:
densities = norm(0, sigma).pdf(resid)

O resultado é um conjunto de densidades de probabilidade, uma para cada elemento do conjunto de dados; o seu produto é a probabilidade dos dados.

In [33]:
likelihood = densities.prod()
likelihood

Como vimos no capítulo anterior, a probabilidade de qualquer conjunto de dados em particular tende a ser pequena.

Se for demasiado pequeno, podemos exceder os limites da aritmética de ponto flutuante.

Quando isso acontece, podemos evitar o problema através do cálculo das probabilidades sob uma transformação de registo.

Mas, neste exemplo, isso não é necessário.

## A Actualização

Agora estamos prontos para fazer a actualização.  Primeiro, precisamos de calcular a probabilidade dos dados para cada conjunto possível de parâmetros.

In [34]:
likelihood = prior.copy()

for slope, inter, sigma in prior.index:
    expected = slope * xs + inter
    resid = ys - expected
    densities = norm.pdf(resid, 0, sigma)
    likelihood[slope, inter, sigma] = densities.prod()

Este cálculo leva mais tempo do que muitos dos exemplos anteriores.

Estamos a aproximar-nos do limite do que podemos fazer com as aproximações da grelha.

No entanto, podemos fazer a actualização da forma habitual:

In [35]:
posterior = prior * likelihood
posterior.normalize()

O resultado é um `Pmf` com um índice de três níveis contendo valores de `slope`, `inter`, e `sigma`.

Para obter as distribuições marginais a partir da articulação posterior, podemos utilizar `Pmf.marginal`, que vimos em <<_ThreeParameterModel>>>>.

In [36]:
posterior_slope = posterior.marginal(0)
posterior_inter = posterior.marginal(1)
posterior_sigma = posterior.marginal(2)

Aqui está a distribuição posterior do `sigma`:

In [37]:
posterior_sigma.plot()

decorate(xlabel='$\sigma$, standard deviation of $\epsilon$',
         ylabel='PDF',
         title='Posterior marginal distribution of $\sigma$')

Os valores mais prováveis para o `sigma` são próximos dos 26 polegadas, o que é consistente com a nossa estimativa baseada no desvio padrão dos dados.

No entanto, para dizer se a queda de neve está a aumentar ou a diminuir, não nos importamos realmente com o `sigma`.  É um "parâmetro de perturbação", assim chamado porque temos de o estimar como parte do modelo, mas não precisamos dele para responder às questões que nos interessam.

No entanto, é bom verificar as distribuições marginais para ter a certeza 

* A localização é consistente com as nossas expectativas, e 

* As probabilidades posteriores estão próximas de 0 nos extremos do intervalo, o que indica que a distribuição anterior abrange todos os parâmetros com probabilidades não negligenciáveis.

Neste exemplo, a distribuição posterior do `sigma` parece bem.

Aqui está a distribuição posterior do `inter`:

In [38]:
posterior_inter.plot(color='C1')
decorate(xlabel='intercept (inches)',
         ylabel='PDF',
         title='Posterior marginal distribution of intercept')

In [39]:
from utils import summarize
    
summarize(posterior_inter) 

A média posterior é de cerca de 64 polegadas, que é a quantidade de neve esperada durante o ano, a meio do intervalo, 1994.

E finalmente, aqui está a distribuição posterior do "declive":

In [40]:
posterior_slope.plot(color='C4')
decorate(xlabel='Slope (inches per year)',
         ylabel='PDF',
         title='Posterior marginal distribution of slope')

In [41]:
summarize(posterior_slope)

A média posterior é de cerca de 0,51 polegadas, o que é consistente com a estimativa que obtivemos a partir da regressão menos quadrática.  

O intervalo credível de 90% é de 0,1 a 0,9, o que indica que a nossa incerteza sobre esta estimativa é bastante elevada.  De facto, existe ainda uma pequena probabilidade posterior (cerca de 2\%) de a inclinação ser negativa. 

In [42]:
posterior_slope.make_cdf()(0)

No entanto, é mais provável que a minha conjectura estivesse errada: estamos realmente a receber mais neve por aqui do que costumávamos receber, aumentando a uma taxa de cerca de meia polegada por ano, o que é substancial.  Em média, obtemos mais 25 polegadas de neve por ano do que quando eu era jovem.

Este exemplo mostra que, com tendências lentas e dados ruidosos, os seus instintos podem ser enganadores.  

Agora, pode-se suspeitar que eu sobrestimava a quantidade de neve quando era jovem porque gostava dela, e subestimo-a agora porque não a tenho.  Mas estaria enganado.

Durante a nevasca de 1978, não tínhamos um limpa-neves e eu e o meu irmão tivemos de cavar a pá.  A minha irmã recebeu um passe sem qualquer razão válida.  O nosso caminho de entrada tinha cerca de 60 pés de comprimento e três carros de largura perto da garagem.  E tivemos de escavar também a entrada do Sr. Crocker, pelo que não nos foi permitido aceitar o pagamento.  Além disso, se bem me lembro, foi durante esta escavação que bati acidentalmente com uma pá na cabeça do meu irmão, e sangrou muito porque, sabe, feridas no couro cabeludo.

De qualquer modo, a questão é que não acho que sobrestime a quantidade de neve quando era jovem, porque tenho boas recordações dela. 

## Optimização

A forma como calculámos a probabilidade na secção anterior foi bastante lenta.  O problema é que fizemos looping em todos os conjuntos de parâmetros possíveis na distribuição anterior, e havia mais de 60.000 deles.

Se conseguirmos fazer mais trabalho por iteração, e correr o laço menos vezes, esperamos que vá mais rápido.

Para o fazer, vou desmarcar a distribuição prévia:

In [43]:
joint3 = prior.unstack()
joint3.head(3)

O resultado é um "DataFrame" com "declive" e "intercepção" ao longo das linhas e "sigmas" ao longo das colunas.

O seguinte é uma versão de ``regressão de probabilidade_regressão` que toma a distribuição prévia conjunta nesta forma e devolve a distribuição posterior na mesma forma.

In [44]:
from utils import normalize

def update_optimized(prior, data):
    """Posterior distribution of regression parameters
    `slope`, `inter`, and `sigma`.
    
    prior: Pmf representing the joint prior
    data: DataFrame with columns `x` and `y`
    
    returns: Pmf representing the joint posterior
    """
    xs = data['x']
    ys = data['y']
    sigmas = prior.columns    
    likelihood = prior.copy()

    for slope, inter in prior.index:
        expected = slope * xs + inter
        resid = ys - expected
        resid_mesh, sigma_mesh = np.meshgrid(resid, sigmas)
        densities = norm.pdf(resid_mesh, 0, sigma_mesh)
        likelihood.loc[slope, inter] = densities.prod(axis=1)
        
    posterior = prior * likelihood
    normalize(posterior)
    return posterior

Esta versão percorre todos os pares possíveis de "inclinação" e "inverno", pelo que o laço corre cerca de 2000 vezes.

In [45]:
len(prior_slope) * len(prior_inter)

Cada vez que atravessa o laço, utiliza uma malha de rede para calcular a probabilidade dos dados para todos os valores de `sigma`.  O resultado é uma matriz com uma coluna para cada ponto de dados e uma linha para cada valor de `sigma`.  Levando o produto através das colunas (`eixo=1`), obtém-se a probabilidade dos dados para cada valor de sigma, que atribuímos como uma linha em `risco`.

In [46]:
%time posterior_opt = update_optimized(joint3, data)

Obtemos o mesmo resultado em ambos os sentidos.

In [47]:
np.allclose(posterior, posterior_opt.stack())

Mas esta versão é cerca de 25 vezes mais rápida do que a versão anterior.  

Esta optimização funciona porque muitas funções em NumPy e SciPy são escritas em C, pelo que funcionam rapidamente em comparação com Python.  Se conseguir fazer mais trabalho cada vez que chamar estas funções, e menos tempo a correr o loop em Python, o seu código correrá muitas vezes substancialmente mais rápido.

Nesta versão da distribuição posterior, o "declive" e o "inverno" correm pelas filas e o "igma" corre pelas colunas.  Assim, podemos utilizar o `marginal` para obter a distribuição conjunta posterior de `slope` e `interceptar`.

In [48]:
from utils import marginal

posterior2 = marginal(posterior_opt, 1)
posterior2.head(3)

O resultado é um `Pmf` com duas colunas no índice.

Para a traçar, temos de a desmarcar.

In [49]:
joint_posterior = posterior2.unstack().transpose()
joint_posterior.head(3)

Aqui está o que parece.

In [50]:
from utils import plot_contour

plot_contour(joint_posterior)
decorate(title='Posterior joint distribution of slope and intercept')

As ovais na curva de nível estão alinhadas com os eixos, o que indica que não há correlação entre "inclinação" e "inverno" na distribuição posterior, que é o que esperamos desde que centrámos os valores.

Neste exemplo, a pergunta motivadora é sobre a inclinação da linha, pelo que lhe respondemos olhando para a distribuição posterior da inclinação.

No exemplo seguinte, a questão motivadora é sobre a previsão, por isso vamos utilizar a distribuição posterior conjunta para gerar distribuições preditivas.

## Recorde Mundial da Maratona

Para muitos eventos de corrida, se traçarmos o ritmo recorde mundial ao longo do tempo, o resultado é uma linha recta notável.  As pessoas, [including me](http://allendowney.blogspot.com/2011/04/two-hour-marathon-in-2045.html), têm especulado sobre possíveis razões para este fenómeno.

As pessoas também têm especulado sobre quando, se é que alguma vez, o tempo recorde mundial para a maratona será inferior a duas horas.

(Nota: Em 2019 Eliud Kipchoge correu a maratona em menos de duas horas, o que é um feito espantoso que eu aprecio plenamente, mas por várias razões não contou como um recorde mundial).

Assim, como segundo exemplo de regressão Bayesiana, vamos considerar a progressão recorde mundial para a maratona (para corredores masculinos), estimar os parâmetros de um modelo linear, e utilizar o modelo para prever quando um corredor irá quebrar a barreira das duas horas.  

A célula seguinte descarrega uma página web da Wikipedia que inclui uma tabela de recordes mundiais da maratona, e utiliza Pandas para colocar os dados num `DataFrame'.

In [51]:
url = 'https://en.wikipedia.org/wiki/Marathon_world_record_progression#Men'
tables = pd.read_html(url)
len(tables)

Se isso não funcionar, tornei disponível uma cópia desta página.  As seguintes células descarregam e analisam-na.

In [52]:
#import os

#datafile = 'Marathon_world_record_progression.html'
#download('https://github.com/AllenDowney/ThinkBayes2/raw/master/data/Marathon_world_record_progression.html')

#tables = pd.read_html(datafile)
#len(tables)

A primeira mesa é a que queremos.

In [53]:
table = tables[0]
table.head(3)

Podemos usar Pandas para analisar as datas.

Algumas delas incluem notas que causam problemas de análise, mas o argumento `errors='coerce'' diz a Pandas para preencher datas inválidas com `NaT', que é uma versão de `NaN' que representa "não um tempo". 

In [54]:
table['date'] = pd.to_datetime(table['Date'], errors='coerce')
table['date'].head()

Podemos também usar Pandas para analisar os tempos recorde.

In [55]:
table['time'] = pd.to_timedelta(table['Time'])

E converter os tempos em ritmos em quilómetros por hora.

In [56]:
table['y'] = 26.2 / table['time'].dt.total_seconds() * 3600
table['y'].head()

A seguinte função traça os resultados.

In [57]:
def plot_speeds(df):
    """Plot marathon world record speed as a function of time.
    
    df: DataFrame with date and mph
    """
    plt.axhline(13.1, color='C5', linestyle='dashed')
    plt.plot(df['date'], df['y'], 'o', 
             label='World record speed', 
             color='C1', alpha=0.5)
    
    decorate(xlabel='Date',
             ylabel='Speed (mph)')

Eis como são os resultados.

A linha tracejada mostra a velocidade necessária para uma maratona de duas horas, 13,1 milhas por hora.

In [58]:
plot_speeds(table)

Não é uma linha perfeitamente direita.  Nos primeiros anos da maratona, a velocidade recorde aumentou rapidamente; desde cerca de 1970, tem vindo a aumentar mais lentamente.

Para a nossa análise, concentremo-nos na progressão recente, iniciada em 1970.

In [59]:
recent = table['date'] > pd.to_datetime('1970')
data = table.loc[recent].copy()
data.head()

No caderno para este capítulo, pode ver como carreguei e limpei os dados.  O resultado é um "DataFrame" que contém as seguintes colunas (e informações adicionais que não utilizaremos):

* "data", que é um Pandas "timestamp" representando a data em que o recorde mundial foi quebrado, e

* "velocidade", que regista o ritmo de quebra de recordes em mph.

Eis como são os resultados, começando em 1970:

In [60]:
plot_speeds(data)

Os pontos de dados caem aproximadamente numa linha, embora seja possível que o declive esteja a aumentar.

Para preparar os dados para a regressão, vou subtrair o ponto médio aproximado do intervalo de tempo, 1995.

In [61]:
offset = pd.to_datetime('1995')
timedelta = table['date'] - offset

Quando subtraímos dois objectos `Timestamp', o resultado é um "delta do tempo", que podemos converter em segundos e depois em anos.

In [62]:
data['x'] = timedelta.dt.total_seconds() / 3600 / 24 / 365.24

In [63]:
data['x'].describe()

Tal como no exemplo anterior, utilizarei a regressão dos mínimos quadrados para calcular as estimativas de pontos para os parâmetros, o que ajudará na escolha dos priores.

In [64]:
import statsmodels.formula.api as smf

formula = 'y ~ x'
results = smf.ols(formula, data=data).fit()
results.params

A intercepção estimada é de cerca de 12,5 mph, que é o ritmo recorde mundial interpolado para 1995.  A inclinação estimada é de cerca de 0,015 mph por ano, que é a taxa que o ritmo recorde mundial está a aumentar, de acordo com o modelo.

Mais uma vez, podemos utilizar o desvio padrão dos resíduos como uma estimativa pontual para o `sigma`.

In [65]:
results.resid.std()

Estes parâmetros dão-nos uma boa ideia de onde devemos colocar as distribuições anteriores.

## Os Priores

Aqui estão as distribuições anteriores que escolhi para "inclinação", "intercepção", e "sigma".

In [66]:
qs = np.linspace(0.012, 0.018, 51)
prior_slope = make_uniform(qs, 'Slope')

In [67]:
qs = np.linspace(12.4, 12.5, 41)
prior_inter = make_uniform(qs, 'Intercept')

In [68]:
qs = np.linspace(0.01, 0.21, 31)
prior_sigma = make_uniform(qs, 'Sigma')

E aqui está a distribuição prévia conjunta.

In [69]:
prior = make_joint3(prior_slope, prior_inter, prior_sigma)
prior.head()

Agora podemos calcular as probabilidades, como no exemplo anterior:

In [70]:
xs = data['x']
ys = data['y']
likelihood = prior.copy()

for slope, inter, sigma in prior.index:
    expected = slope * xs + inter
    resid = ys - expected
    densities = norm.pdf(resid, 0, sigma)
    likelihood[slope, inter, sigma] = densities.prod()

Agora podemos fazer a actualização da forma habitual.

In [71]:
posterior = prior * likelihood
posterior.normalize()

E desembalar os marginais:

In [72]:
posterior_slope = posterior.marginal(0)
posterior_inter = posterior.marginal(1)
posterior_sigma = posterior.marginal(2)

In [73]:
posterior_sigma.plot();

Aqui está a distribuição posterior do `inter`:

In [74]:
posterior_inter.plot(color='C1')
decorate(xlabel='intercept',
         ylabel='PDF',
         title='Posterior marginal distribution of intercept')

In [75]:
summarize(posterior_inter)

A média posterior é de cerca de 12,5 mph, que é o ritmo de maratona recorde mundial que o modelo prevê para o ponto médio do intervalo de datas, 1994.

E aqui está a distribuição posterior do "declive".

In [76]:
posterior_slope.plot(color='C4')
decorate(xlabel='Slope',
         ylabel='PDF',
         title='Posterior marginal distribution of slope')

In [77]:
summarize(posterior_slope)

A média posterior é de cerca de 0,015 mph por ano, ou 0,15 mph por década.

Isso é interessante, mas não responde à pergunta que nos interessa: Quando é que vai haver uma maratona de duas horas? Para responder a isso, temos de fazer previsões.

## Predição

Para gerar previsões, vou retirar uma amostra da distribuição posterior dos parâmetros, depois utilizar a equação de regressão para combinar os parâmetros com os dados.

O "Pmf" fornece "escolha", que podemos utilizar para retirar uma amostra aleatória com substituição, utilizando as probabilidades posteriores como pesos.

In [78]:
np.random.seed(17)

In [79]:
sample = posterior.choice(101)

O resultado é um conjunto de tuplos.  Ao percorrer a amostra, podemos utilizar a equação de regressão para gerar previsões para uma gama de `xs`.

In [80]:
xs = np.arange(-25, 50, 2)
pred = np.empty((len(sample), len(xs)))

for i, (slope, inter, sigma) in enumerate(sample):
    epsilon = norm(0, sigma).rvs(len(xs))
    pred[i] = inter + slope * xs + epsilon

Cada previsão é um conjunto com o mesmo comprimento que "xs", que guardo como uma fila em "pred".  Assim, o resultado tem uma linha para cada amostra e uma coluna para cada valor de `x`.

Podemos utilizar `percentile` para calcular os percentis 5, 50 e 95 em cada coluna.

In [81]:
low, median, high = np.percentile(pred, [5, 50, 95], axis=0)

Para mostrar os resultados, vou traçar a mediana das previsões como uma linha e o intervalo de 90% credível como uma área sombreada.

In [82]:
times = pd.to_timedelta(xs*365.24, unit='days') + offset

plt.fill_between(times, low, high, 
                 color='C2', alpha=0.1)
plt.plot(times, median, color='C2')

plot_speeds(data)

A linha tracejada mostra o ritmo da maratona de duas horas, que é de 13,1 milhas por hora.

Visualmente, podemos estimar que a linha de predição atinge o ritmo alvo entre 2030 e 2040.

Para tornar isto mais preciso, podemos utilizar a interpolação para ver quando as previsões atravessam a linha de chegada.  SciPy fornece `interp1d`, que faz interpolação linear por defeito.

In [83]:
from scipy.interpolate import interp1d

future = np.array([interp1d(high, xs)(13.1),
                   interp1d(median, xs)(13.1),
                   interp1d(low, xs)(13.1)])

In [84]:
dts = pd.to_timedelta(future*365.24, unit='day') + offset
pd.DataFrame(dict(datetime=dts),
             index=['early', 'median', 'late'])

A previsão mediana é 2036, com um intervalo credível de 90% de 2032 a 2043.  Portanto, há cerca de 5% de probabilidade de vermos uma maratona de duas horas antes de 2032.

## Resumo

Este capítulo introduz a regressão Bayesiana, que se baseia no mesmo modelo da regressão de mínimos quadrados; a diferença é que produz uma distribuição posterior para os parâmetros em vez de estimativas pontuais.

No primeiro exemplo, analisámos as mudanças na queda de neve no condado de Norfolk, Massachusetts, e concluímos que agora temos mais queda de neve do que quando eu era jovem, contrariamente às minhas expectativas.

No segundo exemplo, analisámos a progressão do ritmo recorde mundial da maratona masculina, calculámos a distribuição posterior conjunta dos parâmetros de regressão, e utilizámo-la para gerar previsões para os próximos 20 anos.

Estes exemplos têm três parâmetros, pelo que leva um pouco mais de tempo a calcular a probabilidade dos dados.

Com mais de três parâmetros, torna-se impraticável a utilização de algoritmos de grelha.  

Nos próximos capítulos, exploraremos outros algoritmos que reduzam a quantidade de cálculos necessários para fazer uma actualização Bayesiana, o que torna possível a utilização de modelos com mais parâmetros.

Mas primeiro, talvez queira trabalhar nestes exercícios.

## Exercícios



**Exercício:** Tenho a impressão de que é mais quente por aqui do que costumava ser.  Neste exercício, pode pôr à prova as minhas conjecturas.

Utilizaremos o mesmo conjunto de dados que utilizámos para modelar a queda de neve; inclui também temperaturas baixas e altas diárias no Condado de Norfolk, Massachusetts, durante a minha vida.

Aqui estão os dados.

In [85]:
df = pd.read_csv('2239075.csv', parse_dates=[2])
df.head(3)

Mais uma vez, vou criar uma coluna que contém a parte do ano das datas.

In [86]:
df['YEAR'] = df['DATE'].dt.year

Este conjunto de dados inclui `TMIN` e `TMAX`, que são as temperaturas baixas e altas diárias em graus F.

Vou criar uma nova coluna com o ponto médio diário das temperaturas baixas e altas.

In [87]:
df['TMID'] = (df['TMIN'] + df['TMAX']) / 2

Agora podemos agrupar por ano e calcular a média destas temperaturas diárias.

In [88]:
tmid = df.groupby('YEAR')['TMID'].mean()
len(tmid)

Mais uma vez, vou abandonar o primeiro e último anos, que estão incompletos.

In [89]:
complete = tmid.iloc[1:-1]
len(complete)

Eis como se parece a série cronológica.

In [90]:
complete.plot(style='o', alpha=0.5)

decorate(xlabel='Year',
         ylabel='Annual average of daily temperature (deg F)')

Tal como fizemos com os dados da neve, vou converter a "Série" em "DataFrame" para a preparar para a regressão.

In [91]:
data = complete.reset_index()
data.head()

In [92]:
offset = data['YEAR'].mean().round()
offset

In [93]:
data['x'] = data['YEAR'] - offset
data['x'].mean()

In [94]:
data['y'] = data['TMID']
data['y'].std()

Agora podemos usar StatsModels para estimar os parâmetros.

In [95]:
import statsmodels.formula.api as smf

formula = 'y ~ x'
results = smf.ols(formula, data=data).fit()
results.params

E calcular o desvio padrão dos parâmetros.

In [96]:
results.resid.std()

Segundo o modelo de regressão de mínimos quadrados, a temperatura média anual está a aumentar cerca de 0,044 graus F por ano.

Para quantificar a incerteza destes parâmetros e gerar previsões para o futuro, podemos utilizar a regressão Bayesiana.

1. Usar StatsModels para gerar estimativas pontuais para os parâmetros de regressão.

2. Escolher os priores para `slope`, `intercepção`, e `sigma` com base nestas estimativas, e utilizar `make_joint3` para fazer uma distribuição prévia conjunta.

3. Calcular a probabilidade dos dados e calcular a distribuição posterior dos parâmetros.

4. Extrair a distribuição posterior da "inclinação".  Quão confiantes estamos de que a temperatura está a aumentar?

5. Retirar uma amostra de parâmetros da distribuição posterior e utilizá-la para gerar previsões até 2067.

6. Traçar a mediana das previsões e um intervalo credível de 90%, juntamente com os dados observados.  

O modelo encaixa bem nos dados?  Quanto é que esperamos que as temperaturas médias anuais aumentem ao longo da minha vida (esperada)?

In [97]:
# Solution goes here

In [98]:
# Solution goes here

In [99]:
# Solution goes here

In [100]:
# Solution goes here

In [101]:
# Solution goes here

In [102]:
# Solution goes here

In [103]:
# Solution goes here

In [104]:
# Solution goes here

In [105]:
# Solution goes here

In [106]:
# Solution goes here

In [107]:
# Solution goes here

In [108]:
# Solution goes here

In [109]:
# Solution goes here

In [110]:
# Solution goes here

In [111]:
# Solution goes here